In [1]:
from __future__ import annotations

import gymnasium as gym
from PPOA import PPOAgent
import matplotlib.pyplot as plt
import numpy as np
import wandb
import tensorflow as tf
from collections import deque

In [2]:
n_steps = 1000
epsilon = 0.2
frame_count = 0
batch_size = 32

wandb.init(name="PPOAgent_Asteroids",project="deep_rl")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rullofederico16 (fede-). Use `wandb login --relogin` to force relogin


In [3]:
env = gym.make("ALE/Asteroids-v5",render_mode="human")
env = gym.wrappers.AtariPreprocessing(env, frame_skip=1) #Frame preprocessing, resize frame to 84x84, following "Play Atari with DeepRL"
env = gym.wrappers.FrameStack(env,4) #Stack frames into groups of 4
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=5)

In [4]:
agent = PPOAgent(env.action_space.n)

In [5]:
episode_reward_history = []
done = False
episode_count=0
batch_obs = []
batch_actions = []
batch_old_probs = []
batch_advantages = []
batch_returns = []
while True:
    episode_reward = 0

    obs, done = env.reset()
    obs = np.array(obs)
    done = False
    for episode in range(1,n_steps):
        frame_count+=1
        
        action,old_prob = agent.act(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        
        advantage = agent.get_advantage(reward,done,next_obs,obs)
        obs = agent.convert_to_tensor_observations(obs)
        returns = advantage + agent.critic.predict(obs)
        done = terminated or truncated

        batch_obs.append(obs)
        batch_actions.append(tf.one_hot(action,env.action_space.n))
        batch_old_probs.append(old_prob)
        batch_advantages.append(advantage)
        batch_returns.append(returns)
        
        if len(batch_obs) >= batch_size:
            agent.train(batch_obs,batch_actions, batch_old_probs, batch_advantages, batch_returns)
            batch_obs.clear()
            batch_actions.clear()
            batch_old_probs.clear()
            batch_advantages.clear()
            batch_returns.clear()

        obs = next_obs
        episode_reward += reward

    
    episode_reward_history.append(episode_reward)

    if len(episode_reward_history)>100:
        del episode_reward_history[:1]

    running_reward = np.mean(episode_reward_history)

    episode_count += 1
    print(running_reward)
    if len(env.return_queue) != 0 and len(env.length_queue) != 0:
        wandb.log({'Episode Rewards:':np.array(env.return_queue).flatten()[-1],'Episode Lengths':np.array(env.length_queue).flatten()[-1],'Episode Running Rewards:':running_reward})
    else:
        wandb.log({'Episode Running Rewards:':running_reward})
    if running_reward > 800:
        print('Solved at episode{}'.format(episode_count))
        break

1/1 [==============================] - 0s 12ms/step
310.0
1/1 [==============================] - 0s 13ms/step
225.0
1/1 [==============================] - 0s 12ms/step
253.33333333333334
1/1 [==============================] - 0s 12ms/step
297.5
1/1 [==============================] - 0s 12ms/step
282.0
1/1 [==============================] - 0s 12ms/step
